<a href="https://colab.research.google.com/github/xavierruth/DesafioFinal_Residencia_PortoDigital_VacinaCovid/blob/main/VacinaCOVID_Recife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bronze

In [ ]:
%%bash
# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,090 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,464 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,908 kB]
Hit:13 https://ppa.launchpadcontent.net/ubu

In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

In [ ]:
# Definindo o ambiente Spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
%%bash
# Baixando dados com nomes únicos
curl -o vacinados_21.csv http://dados.recife.pe.gov.br/dataset/7ccb3816-0d62-49e1-b39a-3159870883b0/resource/99b42b09-95af-47de-8411-ab99c380c3ef/download/vacinados.csv
curl -o vacinados_22.csv http://dados.recife.pe.gov.br/dataset/7ccb3816-0d62-49e1-b39a-3159870883b0/resource/9664de94-9f07-4adc-848d-b6ef56510762/download/vacinados.csv
curl -o vacinados_23.csv http://dados.recife.pe.gov.br/dataset/7ccb3816-0d62-49e1-b39a-3159870883b0/resource/ca7fb968-3a2c-44ff-a2e8-730d1a689407/download/vacinados.csv


In [ ]:
!ls -lh vacinados_*.csv

In [ ]:
# Verificando se os arquivos existem
import os
print(os.listdir('.'))

In [ ]:
# Carregando os dados no PySpark com o delimitador correto
df1 = spark.read.option("delimiter", ";").csv('vacinados_21.csv', header=True, inferSchema=True)
df2 = spark.read.option("delimiter", ";").csv('vacinados_22.csv', header=True, inferSchema=True)
df3 = spark.read.option("delimiter", ";").csv('vacinados_23.csv', header=True, inferSchema=True)

In [ ]:
# Exibindo os dados para verificar a leitura
df1.show()
df2.show()
df3.show()

In [ ]:
# Verificar o esquema dos DataFrames

df1.printSchema()
df2.printSchema()
df3.printSchema()

In [ ]:
# Função para contar valores nulos
def count_nulls(df):
    return df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
# Função para contar valores em branco
def count_blanks(df):
    return df.select([f.count(f.when(f.col(c) == "", c)).alias(c) for c in df.columns]).show()

In [ ]:
# Contar valores nulos e em branco em vacinados_2021
print("Valores nulos em vacinados_21:")
count_nulls(df1)
print("Valores em branco em vacinados_21:")
count_blanks(df1)

In [ ]:
# Contar valores nulos e em branco em vacinados_2022
print("Valores nulos em vacinados_22:")
count_nulls(df2)
print("Valores em branco em vacinados_22:")
count_blanks(df2)

In [ ]:
# Contar valores nulos e em branco em vacinados_2023
print("Valores nulos em vacinados_23:")
count_nulls(df3)
print("Valores em branco em vacinados_23:")
count_blanks(df3)

In [ ]:
# Salvando os DataFrames em formato Parquet
df1.write.parquet('bronze/vacinados_21.parquet')
df2.write.parquet('bronze/vacinados_22.parquet')
df3.write.parquet('bronze/vacinados_23.parquet')

In [ ]:
# Confirmando a criação dos arquivos Parquet
print("Arquivos Parquet criados:")
print(os.listdir('.'))

# Silver

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, lower

In [ ]:
# Iniciar sessão Spark
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carregar os dados do estágio Bronze
bronze_df1 = spark.read.parquet('bronze/vacinados_21.parquet')
bronze_df2 = spark.read.parquet('bronze/vacinados_22.parquet')
bronze_df3 = spark.read.parquet('bronze/vacinados_23.parquet')


In [ ]:
# Exibindo os dados para verificar a leitura
df1.show()
df2.show()
df3.show()

In [ ]:
# Verificar o esquema dos DataFrames

df1.printSchema()
df2.printSchema()
df3.printSchema()

In [ ]:
# Remover linhas onde qualquer valor é nulo
df1_cleaned = df1.dropna()
df2_cleaned = df2.dropna()
df3_cleaned = df3.dropna()

In [ ]:
# Exibir os DataFrames limpos
df1_cleaned.show()
df2_cleaned.show()
df3_cleaned.show()

In [ ]:
# Verificar o esquema dos DataFrames limpos
df1_cleaned.printSchema()
df2_cleaned.printSchema()
df3_cleaned.printSchema()

In [ ]:
# Função para contar valores nulos
def count_nulls(df):
    return df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

print("Valores nulos em df1_cleaned:")
count_nulls(df1_cleaned)

print("Valores nulos em df2_cleaned:")
count_nulls(df2_cleaned)

print("Valores nulos em df3_cleaned:")
count_nulls(df3_cleaned)

In [ ]:
# Exibir os DataFrames limpos
print("Dados limpos de vacinados_21.csv:")
df1_cleaned.show(15)
print("Dados limpos de vacinados_22.csv:")
df2_cleaned.show(15)
print("Dados limpos de vacinados_23.csv:")
df3_cleaned.show(15)


In [ ]:
# Função para verificar duplicatas em um DataFrame
def check_duplicates(df, df_name):
    duplicates = df.groupBy(df.columns).count().filter(col("count") > 1)
    num_duplicates = duplicates.count()
    if num_duplicates > 0:
        print(f"O DataFrame {df_name} tem {num_duplicates} registros duplicados.")
        duplicates.show()
    else:
        print(f"O DataFrame {df_name} não tem registros duplicados.")

In [ ]:
# Verificar duplicatas antes de combinar
check_duplicates(bronze_df1, "bronze_df1")
check_duplicates(bronze_df2, "bronze_df2")
check_duplicates(bronze_df3, "bronze_df3")

***Vacinados em 2021 relações dentro das tabelas***

In [ ]:
# Registrar a df1 como uma tabela temporária para uso no Spark SQL
df1.createOrReplaceTempView("df1")

Inner Join - saber qual as 4 vacinas mais ministradas em 2021 e as datas com mais vacinações

Com base nesse inner join podemos avaliar qual vacina mais aplicada e o gênero que mais se vacinou em 2021

In [ ]:
# Qual a vacina, o seu total e sexo mais frequente em 2021?
join_query_1 = """
SELECT t.vacina_fabricante,
       t.total_vacinacoes,
       d.sexo_mais_frequente
FROM (
    SELECT vacina_fabricante, COUNT(*) AS total_vacinacoes,
           ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank
    FROM df1
    GROUP BY vacina_fabricante
) t
JOIN (
    SELECT vacina_fabricante, sexo AS sexo_mais_frequente
    FROM (
        SELECT vacina_fabricante, sexo, COUNT(*) AS total_vacinacoes,
               ROW_NUMBER() OVER (PARTITION BY vacina_fabricante ORDER BY COUNT(*) DESC) AS rank
        FROM df1
        GROUP BY vacina_fabricante, sexo
    ) sub
    WHERE sub.rank = 1
) d
ON t.vacina_fabricante = d.vacina_fabricante
WHERE t.rank <= 4
ORDER BY t.total_vacinacoes desc
"""


In [ ]:
# Exibir o resultado 1
df_join_1 = spark.sql(join_query_1)
df_join_1.show()


Concatenação, soma, agrupar e ordenar - saber qual o gênero que mais se vacinou no ano de 2021

In [ ]:
# Qual os dados por faixa-etaria, sexo, raça, grupo que essas pessoas que se vacinaram em 2021 representam?
join_query_2 = """
SELECT faixa_etaria, sexo, raca_cor, grupo,
       COUNT(*) AS total_por_genero,
       CONCAT(ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2), '%') AS percentual
FROM df1
GROUP BY faixa_etaria, sexo, raca_cor, grupo
ORDER BY total_por_genero DESC
"""


In [ ]:
# Exibir o resultado 2
df_join_2 = spark.sql(join_query_2)
df_join_2.show()


Join 3 - Total de grupos por datas que se vacinaram

In [ ]:
# Consulta SQL para calcular as top 10 combinações de município, raça, sexo e grupo mais frequentes, com o percentual por município e ordenação por rank
top_combinacoes_query = """
WITH total_combinacoes AS (
    SELECT
        municipio AS municipio_total,
        raca_cor AS raca_cor_total,
        sexo AS sexo_total,
        COUNT(*) AS total_combinacoes
    FROM
        df1
    WHERE
        municipio IS NOT NULL
        AND raca_cor IS NOT NULL
        AND sexo IS NOT NULL
    GROUP BY
        municipio, raca_cor, sexo
)
SELECT
    df1.municipio AS municipio_df1,
    df1.raca_cor AS raca_cor_df1,
    df1.sexo,
    df1.grupo,
    df1.data_vacinacao,
    CONCAT(ROUND(COUNT(*) * 100.0 / t.total_combinacoes, 2), '%') AS percentual,
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank
FROM
    df1
JOIN
    total_combinacoes t
ON
    df1.municipio = t.municipio_total
    AND df1.raca_cor = t.raca_cor_total
    AND df1.sexo = t.sexo_total
GROUP BY
    municipio_df1, raca_cor_df1, sexo, grupo, data_vacinacao, t.total_combinacoes
ORDER BY
    rank
LIMIT 10
"""

# Executar a consulta SQL para obter as top 10 combinações
df_top_combinacoes = spark.sql(top_combinacoes_query)
df_top_combinacoes.show()


***Vacinados em 2022 relação entre as tabelas***

In [ ]:
# Registrar a df2 como uma tabela temporária para uso no Spark SQL
df2.createOrReplaceTempView("df2")


In [ ]:
# Qual a vacina, o seu total e sexo mais frequente em 2022?
join_df2 = """
SELECT t.vacina_fabricante,
       t.total_vacinacoes,
       d.sexo_mais_frequente
FROM (
    SELECT vacina_fabricante, COUNT(*) AS total_vacinacoes,
           ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank
    FROM df2
    GROUP BY vacina_fabricante
) t
JOIN (
    SELECT vacina_fabricante, sexo AS sexo_mais_frequente
    FROM (
        SELECT vacina_fabricante, sexo, COUNT(*) AS total_vacinacoes,
               ROW_NUMBER() OVER (PARTITION BY vacina_fabricante ORDER BY COUNT(*) DESC) AS rank
        FROM df2
        GROUP BY vacina_fabricante, sexo
    ) sub
    WHERE sub.rank = 1
) d
ON t.vacina_fabricante = d.vacina_fabricante
WHERE t.rank <= 4
ORDER BY t.total_vacinacoes desc
"""

In [ ]:
# Exibir o resultado 2
df_join_df2 = spark.sql(join_df2)
df_join_df2.show()

Concatenação, soma, agrupar e ordenar - saber qual o gênero que mais se vacinou no ano de 2022

In [ ]:
# Qual os dados por faixa-etaria, sexo, raça, grupo que essas pessoas que se vacinaram em 2022 representam?
join_df2_2 = """
SELECT faixa_etaria, sexo, raca_cor, grupo,
       COUNT(*) AS total_por_genero,
       CONCAT(ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2), '%') AS percentual
FROM df2
GROUP BY faixa_etaria, sexo, raca_cor, grupo
ORDER BY total_por_genero DESC
"""

In [ ]:
# Exibir o resultado 2
df_join_df2_2 = spark.sql(join_df2_2)
df_join_df2_2.show()

In [ ]:
# Consulta SQL para calcular as top 10 combinações de município, raça, sexo e grupo mais frequentes, com o percentual por município e ordenação por rank
top_combinacoes_df2 = """
WITH total_combinacoes AS (
    SELECT
        municipio AS municipio_total,
        raca_cor AS raca_cor_total,
        sexo AS sexo_total,
        COUNT(*) AS total_combinacoes
    FROM
        df2
    WHERE
        municipio IS NOT NULL
        AND raca_cor IS NOT NULL
        AND sexo IS NOT NULL
    GROUP BY
        municipio, raca_cor, sexo
)
SELECT
    df2.municipio AS municipio_df2,
    df2.raca_cor AS raca_cor_df2,
    df2.sexo,
    df2.grupo,
    df2.data_vacinacao,
    CONCAT(ROUND(COUNT(*) * 100.0 / t.total_combinacoes, 2), '%') AS percentual,
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank
FROM
    df2
JOIN
    total_combinacoes t
ON
    df2.municipio = t.municipio_total
    AND df2.raca_cor = t.raca_cor_total
    AND df2.sexo = t.sexo_total
GROUP BY
    municipio_df2, raca_cor_df2, sexo, grupo, data_vacinacao, t.total_combinacoes
ORDER BY
    rank
LIMIT 10
"""

In [ ]:
# Executar a consulta SQL para obter as top 10 combinações
top_combinacoes_df2 = spark.sql(top_combinacoes_query)
top_combinacoes_df2.show()

***Combiando as tabelas DF1 e DF2 ***

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, sum

In [ ]:
from pyspark.sql.functions import col

In [ ]:
# Exibir o esquema das DataFrames
print("Esquema de df1:")
df1.printSchema()

print("Esquema de df2:")
df2.printSchema()


In [ ]:
union_df12 = """
SELECT faixa_etaria, sexo, raca_cor, grupo,
       SUM(total_por_genero) AS total_por_genero,
       CONCAT(ROUND(SUM(total_por_genero) * 100.0 / SUM(SUM(total_por_genero)) OVER(), 2), '%') AS percentual
FROM (
    SELECT faixa_etaria, sexo, raca_cor, grupo,
           COUNT(*) AS total_por_genero
    FROM df1
    GROUP BY faixa_etaria, sexo, raca_cor, grupo

    UNION ALL

    SELECT faixa_etaria, sexo, raca_cor, grupo,
           COUNT(*) AS total_por_genero
    FROM df2
    GROUP BY faixa_etaria, sexo, raca_cor, grupo
) combined_data
GROUP BY faixa_etaria, sexo, raca_cor, grupo
ORDER BY total_por_genero DESC
"""

# Executar a consulta SQL para combinar os resultados de df1 e df2
combined_result = spark.sql(union_df12)

# Exibir o resultado
combined_result.show()


In [ ]:
# Registrar as tabelas temporárias para uso no Spark SQL
df1.createOrReplaceTempView("df1")
df2.createOrReplaceTempView("df2")

# Consulta SQL para obter os top 5 registros combinados com base na coluna vacina_fabricante
top_vacina_query = """
SELECT
    vacina_fabricante,
    COUNT(*) AS total_vacinacoes
FROM
    (
    SELECT
        vacina_fabricante
    FROM
        df1
    UNION ALL
    SELECT
        vacina_fabricante
    FROM
        df2
    )
GROUP BY
    vacina_fabricante
ORDER BY
    total_vacinacoes DESC
LIMIT 5
"""

# Executar a consulta SQL
top_vacina_df = spark.sql(top_vacina_query)

# Exibir o resultado
top_vacina_df.show()


In [ ]:
# Salvando os DataFrames limpos em formato Parquet na pasta silver
df1_cleaned.write.parquet('silver/vacinados_21_cleaned.parquet')
df2_cleaned.write.parquet('silver/vacinados_22_cleaned.parquet')
df3_cleaned.write.parquet('silver/vacinados_23_cleaned.parquet')



In [ ]:
# Salvar o DataFrame combinado em formato Parquet na pasta silver
combined_result.write.parquet('silver/combined_vacinados.parquet')


In [ ]:
# Salvar o DataFrame top_vacina_df em formato Parquet na pasta silver
top_vacina_df.write.parquet('silver/top_vacina.parquet')


In [ ]:
# Confirmando a criação dos arquivos Parquet na pasta silver
print("Arquivos Parquet criados na pasta silver:")
print(os.listdir('silver'))

# Gold


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# Iniciar sessão Spark
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carregar os dados da camada Silver
df1_cleaned = spark.read.parquet('silver/vacinados_21_cleaned.parquet')
df2_cleaned = spark.read.parquet('silver/vacinados_22_cleaned.parquet')
df3_cleaned = spark.read.parquet('silver/vacinados_23_cleaned.parquet')
combined_result = spark.read.parquet('silver/combined_vacinados.parquet')
top_vacina_df = spark.read.parquet('silver/top_vacina.parquet')

In [ ]:
# Registrar os DataFrames como tabelas temporárias para uso no Spark SQL
df1_cleaned.createOrReplaceTempView("df1_cleaned")
df2_cleaned.createOrReplaceTempView("df2_cleaned")
df3_cleaned.createOrReplaceTempView("df3_cleaned")

In [ ]:
# Consulta SQL para obter a vacina e o sexo mais frequente para o ano de 2021
gold_query_1 = """
SELECT vacina_fabricante,
       COUNT(*) AS total_vacinacoes,
       sexo AS sexo_mais_frequente
FROM df1_cleaned
GROUP BY vacina_fabricante, sexo
ORDER BY total_vacinacoes DESC
"""


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Executar a consulta SQL
df_gold_1 = spark.sql(gold_query_1)

# Exibir os resultados
df_gold_1.show()

In [ ]:
# Dados fornecidos
vacinas_fabricante = ['3 - COMIRNATY (PFIZER-BIONTECH)', '2 - CHADOX1NCOV-19 (ASTRAZENECA)', '1 - CORONAVAC - SINOVAC', '4 - JANSSEN-COVID-19 (JANSSEN-CILAG)']
total_vacinacoes_feminino = [542301, 381694, 176520, 16420]
total_vacinacoes_masculino = [410535, 302767, 162106, 15389]

# Configuração do gráfico
bar_width = 0.35
index = np.arange(len(vacinas_fabricante))

plt.figure(figsize=(10, 6))

plt.bar(index - bar_width/2, total_vacinacoes_feminino, bar_width, label='Feminino', color='skyblue')
plt.bar(index + bar_width/2, total_vacinacoes_masculino, bar_width, label='Masculino', color='salmon')

# Adicionando rótulos e título
plt.xlabel('Fabricante de Vacina')
plt.ylabel('Total de Vacinações')
plt.title('Total de Vacinações por Fabricante e Sexo em 2021')
plt.xticks(index, vacinas_fabricante, rotation=45)
plt.legend()

# Exibindo o gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Consulta SQL para obter a vacina e o sexo mais frequente para o ano de 2022
gold_query_2 = """
SELECT vacina_fabricante,
       COUNT(*) AS total_vacinacoes,
       sexo AS sexo_mais_frequente
FROM df2_cleaned
GROUP BY vacina_fabricante, sexo
ORDER BY total_vacinacoes DESC
"""


In [ ]:
# Executar a consulta SQL
df_gold_2 = spark.sql(gold_query_2)

# Exibir os resultados
df_gold_2.show()

In [ ]:
# Dados fornecidos
vacinas_fabricante = ['3 - COMIRNATY (PFIZER-BIONTECH)', '2 - CHADOX1NCOV-19 (ASTRAZENECA)', '4 - JANSSEN-COVID-19 (JANSSEN-CILAG)',
                      '5 - COMIRNATY (PFIZER-BIONTECH)', '1 - CORONAVAC - SINOVAC', '6 - CORONAVAC - BUTANTAN', '7 - PFIZER BABY']
total_vacinacoes_feminino = [42614, 23617, 17227, 3280, 1967, 1516, 79]
total_vacinacoes_masculino = [30743, 19500, 12944, 5158, 1917, 1871, 68]

# Configuração do gráfico
bar_width = 0.35
index = np.arange(len(vacinas_fabricante))

plt.figure(figsize=(10, 6))

plt.bar(index - bar_width/2, total_vacinacoes_feminino, bar_width, label='Feminino', color='skyblue')
plt.bar(index + bar_width/2, total_vacinacoes_masculino, bar_width, label='Masculino', color='salmon')

# Adicionando rótulos e título
plt.xlabel('Fabricante de Vacina')
plt.ylabel('Total de Vacinações')
plt.title('Total de Vacinações por Fabricante e Sexo em 2022')
plt.xticks(index, vacinas_fabricante, rotation=45)
plt.legend()

# Exibindo o gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Consulta SQL para obter a vacina e o sexo mais frequente para o ano de 2023
gold_query_3 = """
SELECT vacina_fabricante,
       COUNT(*) AS total_vacinacoes,
       sexo AS sexo_mais_frequente
FROM df3_cleaned
GROUP BY vacina_fabricante, sexo
ORDER BY total_vacinacoes DESC
"""

df_gold_3 = spark.sql(gold_query_3)

In [ ]:
# Executar a consulta SQL
df_gold_3 = spark.sql(gold_query_3)

# Exibir os resultados
df_gold_3.show()

In [ ]:
# Dados fornecidos
vacinas_fabricante = ['3 - COMIRNATY (PFIZER-BIONTECH)', '2 - CHADOX1NCOV-19 (ASTRAZENECA)', '4 - JANSSEN-COVID-19 (JANSSEN-CILAG)',
                      '5 - COMIRNATY (PFIZER-BIONTECH)', '1 - CORONAVAC - SINOVAC', '6 - CORONAVAC - BUTANTAN', '7 - PFIZER BABY']
total_vacinacoes_feminino = [42614, 23617, 17227, 3280, 1967, 1516, 79]
total_vacinacoes_masculino = [30743, 19500, 12944, 5158, 1917, 1871, 68]

# Configuração do gráfico
bar_width = 0.35
index = range(len(vacinas_fabricante))

plt.figure(figsize=(10, 6))

plt.bar(index, total_vacinacoes_feminino, bar_width, label='Feminino', color='skyblue')
plt.bar([i + bar_width for i in index], total_vacinacoes_masculino, bar_width, label='Masculino', color='salmon')

# Adicionando rótulos e título
plt.xlabel('Fabricante de Vacina')
plt.ylabel('Total de Vacinações')
plt.title('Total de Vacinações por Fabricante e Sexo em 2023')
plt.xticks([i + bar_width / 2 for i in index], vacinas_fabricante, rotation=45)
plt.legend()

# Exibindo o gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Consulta SQL para obter os dados por faixa etária, sexo, raça e grupo
gold_query_4 = """
SELECT faixa_etaria, sexo, raca_cor, grupo,
       COUNT(*) AS total_vacinacoes,
       CONCAT(ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2), '%') AS percentual
FROM df1_cleaned
GROUP BY faixa_etaria, sexo, raca_cor, grupo
ORDER BY total_vacinacoes DESC
"""

df_gold_4 = spark.sql(gold_query_4)

In [ ]:
# Executar a consulta SQL
df_gold_4 = spark.sql(gold_query_4)

# Exibir os resultados
df_gold_4.show()

In [ ]:
# Consulta SQL para obter a vacina e o sexo mais frequente combinando todos os anos
gold_query_combined_vacina_sexo = """
SELECT vacina_fabricante,
       COUNT(*) AS total_vacinacoes,
       sexo AS sexo_mais_frequente
FROM (
    SELECT vacina_fabricante, sexo FROM df1_cleaned
    UNION ALL
    SELECT vacina_fabricante, sexo FROM df2_cleaned
    UNION ALL
    SELECT vacina_fabricante, sexo FROM df3_cleaned
)
GROUP BY vacina_fabricante, sexo
ORDER BY total_vacinacoes DESC
"""

In [ ]:
# Executar a consulta SQL
df_gold_combined_vacina_sexo = spark.sql(gold_query_combined_vacina_sexo)


In [ ]:
# Exibir o resultado
df_gold_combined_vacina_sexo.show()

In [ ]:
# Dados fornecidos
data = {
    'vacina_fabricante': ['3 - COMIRNATY (PFIZER)', '2 - CHADOX1NCOV-19 (ASTRAZENECA)', '1 - CORONAVAC - SINOVAC/INSTITUTO BUTANTAN', '4 - JANSSEN-CILAG', '5 - COMIRNATY (PFIZER)', '6 - CORONAVAC - SINOVAC/INSTITUTO BUTANTAN', '7 - PFIZER-BIONTECH'],
    'total_vacinacoes': [627529, 428928, 180454, 50874, 10316, 3742, 294]
}

# Criando o gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(data['vacina_fabricante'], data['total_vacinacoes'], color='skyblue')

# Adicionando rótulos e título
plt.xlabel('Fabricante de Vacina')
plt.ylabel('Total de Vacinações')
plt.title('Total de Vacinações por Fabricante')

# Exibindo o gráfico
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Consulta SQL para obter dados por faixa-etaria, sexo, raça e grupo, combinando todos os anos
gold_query_combined_faixa_etaria = """
SELECT faixa_etaria, sexo, raca_cor, grupo,
       COUNT(*) AS total_por_genero,
       CONCAT(ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2), '%') AS percentual
FROM (
    SELECT faixa_etaria, sexo, raca_cor, grupo FROM df1_cleaned
    UNION ALL
    SELECT faixa_etaria, sexo, raca_cor, grupo FROM df2_cleaned
    UNION ALL
    SELECT faixa_etaria, sexo, raca_cor, grupo FROM df3_cleaned
)
GROUP BY faixa_etaria, sexo, raca_cor, grupo
ORDER BY total_por_genero DESC
"""

In [ ]:
# Executar a consulta SQL
df_gold_combined_faixa_etaria = spark.sql(gold_query_combined_faixa_etaria)

In [ ]:
# Exibir o resultado
df_gold_combined_faixa_etaria.show()

In [ ]:
# Dados fornecidos
faixa_etaria = ['35 a 39 anos', '30 a 34 anos', '25 a 29 anos', '40 a 44 anos', '20 a 24 anos']
total_por_genero_feminino = [45583, 41704, 41161, 40484, 40148]
total_por_genero_masculino = [38432, 35458, 36614, 33425, 36808]

# Configuração do gráfico
bar_width = 0.35
index = range(len(faixa_etaria))

# Criando as barras
plt.figure(figsize=(10, 6))
plt.bar(index, total_por_genero_feminino, bar_width, label='Feminino', color='skyblue')
plt.bar([i + bar_width for i in index], total_por_genero_masculino, bar_width, label='Masculino', color='salmon')

# Adicionando rótulos e título
plt.xlabel('Faixa Etária')
plt.ylabel('Total de Vacinações')
plt.title('Total de Vacinações por Faixa Etária e Gênero')
plt.xticks([i + bar_width / 2 for i in index], faixa_etaria)
plt.legend()

# Exibindo o gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Análise da vacinação por fabricante, sexo, grupo e data da vacinação
gold_query_top_combinacoes = """
WITH total_combinacoes AS (
    SELECT municipio, raca_cor, sexo,
           COUNT(*) AS total_combinacoes
    FROM (
        SELECT municipio, raca_cor, sexo FROM df1_cleaned
        UNION ALL
        SELECT municipio, raca_cor, sexo FROM df2_cleaned
        UNION ALL
        SELECT municipio, raca_cor, sexo FROM df3_cleaned
    )
    WHERE municipio IS NOT NULL AND raca_cor IS NOT NULL AND sexo IS NOT NULL
    GROUP BY municipio, raca_cor, sexo
)
SELECT combined.municipio, combined.raca_cor, combined.sexo, combined.grupo,
       COUNT(*) AS total_por_combinacao,
       CONCAT(ROUND(COUNT(*) * 100.0 / t.total_combinacoes, 2), '%') AS percentual,
       ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank
FROM (
    SELECT municipio, raca_cor, sexo, grupo FROM df1_cleaned
    UNION ALL
    SELECT municipio, raca_cor, sexo, grupo FROM df2_cleaned
    UNION ALL
    SELECT municipio, raca_cor, sexo, grupo FROM df3_cleaned
) combined
JOIN total_combinacoes t
ON combined.municipio = t.municipio AND combined.raca_cor = t.raca_cor AND combined.sexo = t.sexo
GROUP BY combined.municipio, combined.raca_cor, combined.sexo, combined.grupo, t.total_combinacoes
ORDER BY rank
LIMIT 10
"""

In [ ]:
# Executar a consulta SQL
df_gold_top_combinacoes = spark.sql(gold_query_top_combinacoes)

In [ ]:
# Exibir o resultado
df_gold_top_combinacoes.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados fornecidos
combinacoes = ['PARDA_FEMININO_PÚBLICO EM GERAL', 'PARDA_MASCULINO_PÚBLICO EM GERAL', 'BRANCA_FEMININO_PÚBLICO EM GERAL',
               'BRANCA_MASCULINO_PÚBLICO EM GERAL', 'PRETA_MASCULINO_PÚBLICO EM GERAL', 'PRETA_FEMININO_PÚBLICO EM GERAL',
               'BRANCA_FEMININO_IDOSOS', 'BRANCA_FEMININO_PESSOAS COM COMORBIDADES', 'PARDA_FEMININO_PESSOAS COM COMORBIDADES',
               'BRANCA_MASCULINO_PESSOAS COM COMORBIDADES']
total_por_combinacao = [300316, 257595, 204601, 169159, 83606, 82255, 77509, 65420, 63472, 62932]

# Configuração do gráfico
plt.figure(figsize=(10, 6))
plt.barh(combinacoes, total_por_combinacao, color='skyblue')

# Adicionando rótulos e título
plt.xlabel('Total de Vacinações')
plt.ylabel('Combinação de Raça, Sexo e Grupo')
plt.title('Total de Vacinações por Combinação em Recife')

# Exibindo o gráfico
plt.gca().invert_yaxis()  # Inverter eixo y para ter a combinação mais importante no topo
plt.show()

In [ ]:
# Salvar os resultados no estágio Gold
df_gold_1.write.parquet('gold/vacina_sexo_2021.parquet')
df_gold_2.write.parquet('gold/vacina_sexo_2022.parquet')
df_gold_3.write.parquet('gold/vacina_sexo_2023.parquet')
df_gold_4.write.parquet('gold/faixa_etaria_2021.parquet')
df_gold_combined_vacina_sexo.write.parquet('gold/combined_vacina_sexo.parquet')
df_gold_combined_faixa_etaria.write.parquet('gold/combined_faixa_etaria.parquet')
df_gold_top_combinacoes.write.parquet('gold/top_combinacoes.parquet')

In [ ]:
# Confirmando a criação dos arquivos Parquet na pasta Gold
print("Arquivos Parquet criados na pasta gold:")
print(os.listdir('gold'))